In [3]:
import requests
import pandas as pd
from pprint import pprint
import json
from config import api_key
import time
import numpy as np

In [5]:
#Create an empty DataFrame to store the API data
weekly_games_df = pd.DataFrame()

In [6]:
#Create base_url to loop with to pull the season weekly schedule and postseason weekly schedule
base_url = "http://api.sportradar.us/nfl/official/trial/v6/en/games/2020/"
#Empty list to store urls for the .get() request
urls = []
#loop through each week in the season and pull that week's schedule url and append it to the list of urls
for i in range(1,18):
    url = base_url + "REG/"+ str(i) + "/schedule.json?api_key=" + api_key
    urls.append(url)
#loop through each week in the post season and pull that week's schedule url and append it to the list of urls
for i in range(1,5):
    url = base_url + "PST/"+ str(i) + "/schedule.json?api_key=" + api_key
    urls.append(url)

In [7]:
def grab_base_data(response):
    '''Will take a json response from Sport Radar's Weekly Schedule NFL api 
    and return the game ID, game reference, game number, game schedule, game attendance, 
    game weather, game sr_id, game home team name and alias, game away team and alias, 
    final score and scores at the end of each quarter'''
    #empty lists to temporarily pull data for the response
    ids = []
    references = []
    numbers = []
    scheduled = []
    attendances = []
    weather = []
    sr_ids = []
    home_teams = []
    home_team_alias = []
    home_team_id = []
    away_teams = []
    away_team_alias = []
    away_team_id = []
    home_final = []
    away_final = []
    home_q1 = []
    away_q1 = []
    home_q2 = []
    away_q2 = []
    home_q3 = []
    away_q3 = []
    home_q4 = []
    away_q4 = []
    home_OT = []
    away_OT = []
    #loop through each game within the week's schedule and pull the following data
    for game in response['week']['games']:
        #we place each call within a try-except statement so we can add "None" if no info is available within the API
        try:
            ids.append(game['id'])
        except:
            ids.append(None)
        try:
            references.append(game['reference'])
        except:
            references.append(None)
        try:
            numbers.append(game['number'])
        except:
            numbers.append(None)
        try:
            scheduled.append(game['scheduled'])
        except:
            scheduled.append(None)
        try:
            attendances.append(game['attendance'])
        except:
            attendances.append(None)
        try:
            weather.append(game['weather'])
        except:
            weather.append(None)
        try:
            sr_ids.append(game['sr_id'])
        except:
            sr_ids.append(None)
        try:
            home_teams.append(game['home']['name'])
        except:
            home_teams.append(None)
        try:
            home_team_alias.append(game['home']['alias'])
        except:
            home_team_alias.append(None)
        try:
            home_team_id.append(game['home']['id'])
        except:
            home_team_id.append(None)
        try:
            away_teams.append(game['away']['name'])
        except:
            away_teams.append(None)
        try:
            away_team_alias.append(game['away']['alias'])
        except:
            away_team_alias.append(None)
        try:
            away_team_id.append(game['away']['id'])
        except:
            away_team_id.append(None)
        try:
            home_final.append(game['scoring']['home_points'])
            away_final.append(game['scoring']['away_points'])
        except:
            home_final.append(None)
            away_final.append(None)
        try:
            home_q1.append(game['scoring']['periods'][0]['home_points'])
            away_q1.append(game['scoring']['periods'][0]['away_points'])
        except:
            home_q1.append(None)
            away_q1.append(None)
        try:
            home_q2.append(game['scoring']['periods'][1]['home_points'])
            away_q2.append(game['scoring']['periods'][1]['away_points'])
        except:
            home_q2.append(None)
            away_q2.append(None)
        try:
            home_q3.append(game['scoring']['periods'][2]['home_points'])
            away_q3.append(game['scoring']['periods'][2]['away_points'])
        except:
            home_q3.append(None)
            away_q3.append(None)
        try:
            home_q4.append(game['scoring']['periods'][3]['home_points'])
            away_q4.append(game['scoring']['periods'][3]['away_points'])
        except:
            home_q4.append(None)
            away_q4.append(None)
        try:
            home_OT.append(game['scoring']['periods'][4]['home_points'])
            away_OT.append(game['scoring']['periods'][4]['away_points'])
        except:
            home_OT.append(None)
            away_OT.append(None)
    rows = list(zip(ids, references, numbers, scheduled, attendances, weather, sr_ids, 
                    home_teams, home_team_alias, home_team_id, away_teams, away_team_alias, 
                    away_team_id, home_final, away_final, home_q1, away_q1, home_q2, away_q2, 
                    home_q3, away_q3, home_q4, away_q4, home_OT, away_OT))
    return rows



In [8]:
def team_ids(response):
    '''Grabs each team id for every game in the response'''
    team_ids = []
    for game in response['week']['games']:
        team_ids.append(game['home']['id'])
    return team_ids

In [9]:
#Setting up an empty list to store all the team ids
unique_teams = []
#Loop through all urls and grab the response
for url in urls:
    response = requests.get(url).json()
    #adding in a time.sleep statement to force the code to wait before performing another API call
    time.sleep(1)
    #grabbing needed data about the game
    base_data = grab_base_data(response)
    #grabbing all team ids for the url provided
    unique_teams.append(team_ids(response))
    #appending game data into our DataFrame
    weekly_games_df = weekly_games_df.append(base_data)

In [10]:
weekly_games_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,018556f9-1977-4a0b-8244-20cad15df9a4,58178,11,2020-09-13T20:05:00+00:00,0.0,"Partly Cloudy Temp: 83 F, Humidity: 56%, Wind:...",sr:match:21980371,Cincinnati Bengals,CIN,ad4ae08f-d808-42d5-a1e6-e9bc4e34d123,...,7.0,0.0,0.0,6.0,6.0,0.0,0.0,10.0,None,None
1,10339048-8c6b-43c7-ae7c-d68cf44417d3,58183,16,2020-09-15T02:20:00+00:00,0.0,"Mostly Clear Temp: 79 F, Humidity: 14%, Wind: ...",sr:match:21980381,Denver Broncos,DEN,ce92bd47-93d5-4fe9-ada4-0fc681e6caa0,...,7.0,0.0,0.0,7.0,0.0,0.0,7.0,9.0,None,None
2,213b8eda-43d8-4e62-9951-ed4c605fca0d,58175,8,2020-09-13T17:00:00+00:00,0.0,N/A (Indoors) Temp: Humidity: Wind:,sr:match:21980357,Minnesota Vikings,MIN,33405046-04ee-4058-a950-d606f8c30852,...,7.0,3.0,3.0,19.0,0.0,7.0,24.0,14.0,None,None
3,24734cde-bf1b-46e5-ad50-d61e47aeca26,58174,7,2020-09-13T17:00:00+00:00,14100.0,"Partly Cloudy Temp: 80 F, Humidity: 85%, Wind:...",sr:match:21980359,Jacksonville Jaguars,JAC,f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de,...,0.0,7.0,14.0,10.0,3.0,0.0,10.0,3.0,None,None
4,5a5b0a83-1765-4729-b914-73ff7d4c4c89,58172,5,2020-09-13T17:00:00+00:00,0.0,"Mostly Sunny Temp: 81 F, Humidity: 59%, Wind: ...",sr:match:21980367,Carolina Panthers,CAR,f14bf5cc-9a82-4a38-bc15-d39f75ed5314,...,9.0,7.0,6.0,10.0,0.0,10.0,15.0,7.0,None,None


In [11]:
#Rename columns to be readable and descriptive
weekly_games_df.rename(columns = {0: 'Game ID',
                                  1: 'Game Reference Number',
                                  2: 'Game Number',
                                  3: 'Scheduled Time',
                                  4: 'Attendance',
                                  5: 'Weather', 
                                  6: 'Sr_ID',
                                  7: 'Home Team Name',
                                  8: 'Home Team Alias',
                                  9: 'Home Team Id',
                                  10: 'Away Team Name',
                                  11: 'Away Team Alias',
                                  12: 'Away Team Id',
                                  13: 'Final Score (Home)',
                                  14: 'Final Score (Away)',
                                  15: 'Quarter 1 Points (Home)',
                                  16: 'Quarter 1 Points (Away)',
                                  17: 'Quarter 2 Points (Home)',
                                  18: 'Quarter 2 Points (Away)',
                                  19: 'Quarter 3 Points (Home)',
                                  20: 'Quarter 3 Points (Away)',
                                  21: 'Quarter 4 Points (Home)',
                                  22: 'Quarter 4 Points (Away)',
                                  23: 'Overtime Points (Home)',
                                  24: 'Overtime Points (Away)'}, inplace=True)

In [12]:
# test = weekly_games_df[weekly_games_df['Home Team Id']=='6680d28d-d4d2-49f6-aace-5292d3ec02c2']
# test[['Home Team Name', 'Away Team Name', 'Quarter 4 Points (Home)', 'Quarter 4 Points (Away)']]

In [13]:
#Take our unique teams list and flatten it so we can remove duplicates
unique_teams = list(np.concatenate(unique_teams).flat)
unique_teams

['ad4ae08f-d808-42d5-a1e6-e9bc4e34d123',
 'ce92bd47-93d5-4fe9-ada4-0fc681e6caa0',
 '33405046-04ee-4058-a950-d606f8c30852',
 'f7ddd7fa-0bae-4f90-bc8e-669e4d6cf2de',
 'f14bf5cc-9a82-4a38-bc15-d39f75ed5314',
 '6680d28d-d4d2-49f6-aace-5292d3ec02c2',
 '04aa1c9d-66da-489d-b16a-1dee3f2eec4d',
 '2eff2a03-54d4-46ba-890e-2bc3925548f3',
 '768c92aa-75ff-4a43-bcc0-f2798c2e1724',
 'e6aa13a4-0055-48a9-bc41-be28dc106929',
 '22052ff7-c065-42ee-bc8f-c4691c50e624',
 'f0e724b0-4cbf-495a-be47-013907608da9',
 '97354895-8c77-4fd4-a860-32e62ea7382a',
 'ebd87119-b331-4469-9ea6-d51fe3ce2f1c',
 'c5a59daa-53a7-4de0-851f-fb12be893e9e',
 '0d855753-ea21-4953-89f9-0e20aff9eb73',
 'd26a1ca5-722d-4274-8f97-c92e49c96315',
 '7b112545-38e6-483c-a55c-96cf6ee49cb8',
 'a20471b4-a8d9-40c7-95ad-90cc30e46932',
 '4254d319-1bc7-4f81-b4ab-b5e6f3402b69',
 '4809ecb0-abd3-451d-9c4a-92a90b83ca06',
 '5fee86ae-74ab-4bdd-8416-42a9dd9964f3',
 '82d2d380-3834-4938-835f-aec541e5ece7',
 'd5a2eb42-8065-4174-ab79-0a6fa820e35e',
 '7d4fcc64-9cb5-

In [14]:
#checking our length of team ids
len(unique_teams)

271

In [15]:
#Removing duplicates from our ids list 
unique_teams_list = set(unique_teams)
unique_teams_list

{'04aa1c9d-66da-489d-b16a-1dee3f2eec4d',
 '0d855753-ea21-4953-89f9-0e20aff9eb73',
 '1f6dcffb-9823-43cd-9ff4-e7a8466749b5',
 '22052ff7-c065-42ee-bc8f-c4691c50e624',
 '2eff2a03-54d4-46ba-890e-2bc3925548f3',
 '33405046-04ee-4058-a950-d606f8c30852',
 '386bdbf9-9eea-4869-bb9a-274b0bc66e80',
 '3d08af9e-c767-4f88-a7dc-b920c6d2b4a8',
 '4254d319-1bc7-4f81-b4ab-b5e6f3402b69',
 '4809ecb0-abd3-451d-9c4a-92a90b83ca06',
 '5fee86ae-74ab-4bdd-8416-42a9dd9964f3',
 '6680d28d-d4d2-49f6-aace-5292d3ec02c2',
 '768c92aa-75ff-4a43-bcc0-f2798c2e1724',
 '7b112545-38e6-483c-a55c-96cf6ee49cb8',
 '7d4fcc64-9cb5-4d1b-8e75-8a906d1e1576',
 '82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9',
 '82d2d380-3834-4938-835f-aec541e5ece7',
 '97354895-8c77-4fd4-a860-32e62ea7382a',
 'a20471b4-a8d9-40c7-95ad-90cc30e46932',
 'ad4ae08f-d808-42d5-a1e6-e9bc4e34d123',
 'c5a59daa-53a7-4de0-851f-fb12be893e9e',
 'cb2f9f1f-ac67-424e-9e72-1475cb0ed398',
 'ce92bd47-93d5-4fe9-ada4-0fc681e6caa0',
 'd26a1ca5-722d-4274-8f97-c92e49c96315',
 'd5a2eb42-8065-

In [16]:
#checking to make sure we have exactly 32 teams
len(unique_teams_list)

32